In [1]:
import pandas as pd 
import numpy as np
import random
from imblearn.under_sampling import RandomUnderSampler

In [2]:
wine_df = pd.read_csv('../winemag_data_first150k.csv')

In [3]:
wine_df = wine_df.drop_duplicates(subset='description', keep='last')

wine_df = wine_df.drop(['Unnamed: 0', 'country', 'designation', 'points', 'price', 'province', 
                        'region_1', 'region_2', 'winery'], axis=1)

greater = wine_df.variety.value_counts() >= 50
greater_tru = greater[greater==True]
greaterthan50 = list(greater_tru.index)

wine_df.variety = wine_df.variety.apply(lambda x:x if x in greaterthan50 else 'Other')

wine_50_df = wine_df[wine_df.variety!='Other']
wine_50_df['fake_X'] = wine_50_df.index
fake_X = np.array(wine_50_df['fake_X'])

/Users/adamszabunio/anaconda/envs/dl/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]


In [4]:
target = wine_50_df['variety']
y = pd.get_dummies(target)
_, y_scalar = np.where(y>0)

In [5]:
rus2 = RandomUnderSampler(replacement=False)
fake_X = fake_X.reshape(len(fake_X), 1)
X_res, y_res = rus2.fit_sample(fake_X, y_scalar)

X_res.shape, y_res.shape

((5508, 1), (5508,))

In [6]:
X_res[:,0].shape

(5508,)

In [7]:
balanced_lst = X_res[:, 0]
balanced_lst

array([ 28914,  29199,  33708, ..., 127667, 105235, 148147])

In [8]:
wine_50_df.fake_X = wine_50_df.fake_X.apply(lambda x:x if x in balanced_lst else 'out')
wine_50_df.head()

/Users/adamszabunio/anaconda/envs/dl/lib/python3.6/site-packages/pandas/core/generic.py:2999: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


,description,variety,fake_X
0,This tremendous 100% varietal wine hails from ...,Cabernet Sauvignon,out
1,"Ripe aromas of fig, blackberry and cassis are ...",Tinta de Toro,1
2,Mac Watson honors the memory of a wine once ma...,Sauvignon Blanc,out
3,"This spent 20 months in 30% new French oak, an...",Pinot Noir,out
5,"Deep, dense and pure from the opening bell, th...",Tinta de Toro,out


In [9]:
balanced_df = wine_50_df[wine_50_df.fake_X !='out']
balanced_df.head()

,description,variety,fake_X
1,"Ripe aromas of fig, blackberry and cassis are ...",Tinta de Toro,1
6,Slightly gritty black-fruit aromas include a s...,Tinta de Toro,6
10,"Elegance, complexity and structure come togeth...",Friulano,10
13,This wine is in peak condition. The tannins an...,Tannat,13
22,Tarry blackberry and cheesy oak aromas are app...,Tinta de Toro,22


In [10]:
balanced_df.shape

(5508, 3)

In [11]:
variety_text = balanced_df.variety + ": " + balanced_df.description

variety_text.shape

(5508,)

In [12]:
variety_text.head()

1     Tinta de Toro: Ripe aromas of fig, blackberry ...
6     Tinta de Toro: Slightly gritty black-fruit aro...
10    Friulano: Elegance, complexity and structure c...
13    Tannat: This wine is in peak condition. The ta...
22    Tinta de Toro: Tarry blackberry and cheesy oak...
dtype: object

In [18]:
rand_generator = np.random.randint((variety_text.shape[0]-1))
list(variety_text)[rand_generator]

"Chardonnay: While this Chardonnay is packed with ripe pineapple, pear and tropical fruit and has decent acidity, it's awkwardly sugary and raw. Tastes like Chinese sweet and sour sauce. The fruit is good, but the wine needs to be drier."

In [26]:
for i in range(10):
    print(list(variety_text)[np.random.randint((variety_text.shape[0]-1))])
    print()

Sauvignon: Aromas include cut grass, exotic fruit and crushed tomato vine. The palate delivers green melon and creamy white peach brightened by lemon zest and crisp acidity.

Mencía: Dark as night, with prune and rubber on the nose along with murk and compost. It's extracted to the max, leaving a rubbery feel to go with supercharged boysenberry flavors. Gritty and forceful on the finish, and pushing tons of runaway flavor. Not for those who like restrained red wines.

Pinot Bianco: A clean and luminous appearance set the stage for what follows: aromas of citrus, white stone, wildflowers and grass. Crisp and refreshing citrus flavors follow through to the palate and finish.

Pinot Grigio: Despite never having seen a splinter of wood, this wine is hugely rich and enormous in the most palate-flattering flavors. It's an explosion of jammy peaches, Delicious apples, mangoes and kiwi fruits. The acidity is high and the wine is balanced and clean, but it's the amazing fruit that really stars.

# Spot the fake

a) Rhône-style Red Blend: On the lighter side of the Côtes du Rhône spectrum, with a slightly briary note to the cherry fruit. Turns more olivaceous on the silky-textured finish. Drink now.

b) Semillon-Sauvignon Blanc: Slender and simple, this has a green asparagus flavor that thins the citrus and pear notes.

c) Pinot Nero: This is a touch of pie flavors are dried finish. A bit of grilled fruit. The wine is for the hotey.

d) Cabernet Sauvignon-Shiraz: Full-bodied and powerful, this is still quite youthful, powered by Graves-like notes of earth, tobacco and vanilla wrapped around a solid core of cassis. A bit chewy on the finish, but there's plenty of fruit, too; drink 2010–2020. 74% Cabernet Sauvignon, 26% Shiraz.

___

a) Grenache: This Armada vineyard (Cayuse) Grenache has got the chewy flavors of white peach, apple and cola and oak wine. The wine is the interesting style the structure, elegantly pear in the mouth and troping and very example oak black pricolfors.

b) Malbec-Merlot: This is an attractive wine, with fruit and firm, supporting tannins. It has a mint flavor and a smooth texture, balanced with damson fruit. This is ripe and ready to drink over the next year.

c) Malbec: Savory and a step up compared to the rest of Chile's under-$15 Malbecs. This wine has complexity and character; it smells a little like Syrah but in the mouth it's as full, cuddly and friendly as any Malbec from Argentina or elsewhere. Saturated, fruity and fun to drink, with a spot of heat and char on the finish.

d) G-S-M: This starts off with a huge whack of toasty, vanilla-laden oak, then supports those flavors with just enough weight and body. Opt for drinking this over the next 1–2 years, before the fruit drops off.


___

a) Fumé Blanc: Light, pleasant and fruity, with plenty of lemon and honey flavors. It's also nicely dry, with good acidity and balance.

b) Shiraz-Viognier: A medium-bodied wine with gentle tannins, this is a savory offering, with notes of plum, meat, tapenade and peppery spices accented by cedary oak, particularly apparent on the mouth-watering finish. Drink now–2015.

c) Bordeaux-style White Blend: This is a ripe and raspberry and cherry and cherry finish. The wine is lean candied black cherry, cassis, but the close. Turny wine, mouthfeel is a power and structure or a complex, with soft notes of fresh acidity.

d) Agiorgitiko: Blueberry, pepper and cedar aromas and flavors dominate in this structured, elegant red. The wine finishes long with spice and dark cherry notes. A collectible wine that will pair well with poultry, game.


____

a) Meritage: An enticing Meritage, with aromas of violet, anise on the finish. This is a structured and structured that is rich, with a fresh, structured and fresh acidity that shows a soft, pineapple, pepper aromas of citrus and curam, strawberry, plum and ripe value and cola, toasted nut on the palate, with a hint of spice flavors are dried citrus and cassis and a pleasant flavors of the vintage.

b) Mencía: Dark as night, with prune and rubber on the nose along with murk and compost. It's extracted to the max, leaving a rubbery feel to go with supercharged boysenberry flavors. Gritty and forceful on the finish, and pushing tons of runaway flavor. Not for those who like restrained red wines.

c) Mencía: Punchy, raw cranberry and red-plum aromas turn more candied with airing. This has an easy flowing feel as well as simple, fresh cherry and raspberry flavors. Dry spice, cocoa and residual cherry flavors drive the finish.

d) Verdejo: This is fresh and clean upfront, but mild, with saline, mineral and melon aromas. It feels round and mellow, with tame acidity and waxy flavors of peach, melon and citrus. Dry on the finish.


---